<a href="https://colab.research.google.com/github/BiancaNiemann/Market_Flash_Project/blob/main/MarketFlash_Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project='snappy-rainfall-450911-k6')

In [14]:
# Install packages if not already installed (for Colab use)
!pip install --quiet pandas google-cloud-bigquery

In [15]:
from google.cloud import bigquery
import pandas as pd

In [16]:
# Set your project ID
project_id = "snappy-rainfall-450911-k6"

In [17]:
# Initialize client
client = bigquery.Client(project=project_id)

In [18]:
# Total rows in campaigns table (1000)

query = """
SELECT
  COUNT(DISTINCT campaign_id) AS total_campaigns
FROM
  `snappy-rainfall-450911-k6.market_flash_stage.campaigns`;
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,total_campaigns
0,1000


In [19]:
# totals campaigns

query = """
SELECT
  COUNT(DISTINCT campaign_name) AS total_campaigns
FROM
  `snappy-rainfall-450911-k6.market_flash_stage.campaigns`
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,total_campaigns
0,1000


In [20]:
# top 10 rows

query = """
SELECT
  *
FROM
  `snappy-rainfall-450911-k6.market_flash_stage.campaigns`
LIMIT
  10
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,campaign_id,campaign_name,start_date,end_date,campaign_budget,campaign_cost,emp_id,pmt_id,content_id,metric_id,client_id,ad_id,loc_id,aud_id
0,1,Campaign 1,2023-12-18,2024-01-10,17451.2875,13961.03,emp_001,ff111cd7-9a9b-4cff-a713-4a32e044212a,con_001,met_001,cli_001,ad_002,loc_001,aud_001
1,2,Campaign 2,2023-10-12,2023-11-09,54755.3875,43804.31,emp_002,0b415dd8-d6d7-46d1-a79c-213170956edf,con_002,met_002,cli_002,ad_003,loc_002,aud_002
2,3,Campaign 3,2023-05-18,2023-06-04,45009.3375,36007.47,emp_003,9e15fb41-4f9f-4699-a53e-924e4fb22c20,con_003,met_003,cli_003,ad_002,loc_003,aud_003
3,4,Campaign 4,2023-02-23,2023-03-09,46782.3125,37425.85,emp_003,8053dbe1-1ffb-4433-afdf-1b5402563f02,con_004,met_004,cli_004,ad_002,loc_004,aud_004
4,5,Campaign 5,2023-11-20,2023-12-11,60737.9250,48590.34,emp_004,0862cce9-4848-4e3d-a46f-e4bdfef5522f,con_005,met_005,cli_005,ad_003,loc_003,aud_005
5,6,Campaign 6,2023-09-19,2023-10-05,33367.2750,26693.82,emp_004,7480477a-d396-4276-894c-878050e50d44,con_006,met_006,cli_006,ad_003,loc_001,aud_005
6,7,Campaign 7,2023-06-19,2023-07-09,5343.0625,4274.45,emp_003,5d2c557a-a42a-47e8-968d-7b10aede56e7,con_007,met_007,cli_003,ad_004,loc_005,aud_006
7,8,Campaign 8,2023-07-08,2023-07-20,9829.0000,7863.20,emp_002,2e8f9028-1d8f-4de4-8e52-72ea4489b328,con_008,met_008,cli_007,ad_004,loc_006,aud_007
8,9,Campaign 9,2023-04-24,2023-05-17,18006.1875,14404.95,emp_004,a0db8143-0490-41b2-bc2f-d61f29aca56e,con_009,met_009,cli_005,ad_003,loc_007,aud_001
9,10,Campaign 10,2023-06-05,2023-06-07,32549.8500,26039.88,emp_002,7b6ff85c-7d1b-48e4-96a2-08e062c404ce,con_010,met_010,cli_008,ad_003,loc_001,aud_005


In [21]:
# Get avg cost per platform

query = """
SELECT
  pl.platform_name,
  ROUND(AVG(ca.campaign_cost),2) AS average_cost
FROM
  `snappy-rainfall-450911-k6.market_flash_stage.campaigns` ca
LEFT JOIN
  `snappy-rainfall-450911-k6.market_flash_stage.campaign_platform` cp
ON
  ca.campaign_id = cp.campaign_id
LEFT JOIN
  `snappy-rainfall-450911-k6.market_flash_stage.platforms` pl
ON
  cp.platform_Id = pl.platform_id
GROUP BY
  pl.platform_name
ORDER BY
  average_cost DESC
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,platform_name,average_cost
0,Facebook,25808.60
1,YouTube,25622.74
2,Email,24376.61
3,Instagram,24348.14
4,TikTok,24212.23
5,None,NaN


In [22]:
# Avg days per campaign

query = """
SELECT
  ROUND(AVG(DATE_DIFF(end_date, start_date, DAY)),2)
FROM
  `snappy-rainfall-450911-k6.market_flash_stage.campaigns`
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,f0_
0,15.59


In [23]:
#   Create table for intergration with campaign and platform combine

query = """
SELECT
  *
FROM
  `snappy-rainfall-450911-k6.market_flash_stage.campaigns` ca
LEFT JOIN
  `snappy-rainfall-450911-k6.market_flash_stage.campaign_platform` cp
ON
  ca.campaign_id = cp.campaign_id
LEFT JOIN
  `snappy-rainfall-450911-k6.market_flash_stage.platforms` pl
ON
  cp.platform_Id = pl.platform_id
WHERE
  ca.campaign_id IS NOT NULL
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,campaign_id,campaign_name,start_date,end_date,campaign_budget,campaign_cost,emp_id,pmt_id,content_id,metric_id,...,campaign_id_1,platform_Id,platform_id_1,platform_name,url,platform_contact_person,platform_phone,platform_email,ad_id_1,row_num
0,1,Campaign 1,2023-12-18,2024-01-10,17451.2875,13961.03,emp_001,ff111cd7-9a9b-4cff-a713-4a32e044212a,con_001,met_001,...,1,pla_001,pla_001,YouTube,https://www.youtube.com,Sarah Jennings,(555) 123-4567,sarah.jennings@marketflash.com,ad_003,1
1,2,Campaign 2,2023-10-12,2023-11-09,54755.3875,43804.31,emp_002,0b415dd8-d6d7-46d1-a79c-213170956edf,con_002,met_002,...,2,pla_002,pla_002,Email,https://mail.marketflash.com,Jason Moore,(555) 234-5678,jason.moore@marketflash.com,ad_001,2
2,3,Campaign 3,2023-05-18,2023-06-04,45009.3375,36007.47,emp_003,9e15fb41-4f9f-4699-a53e-924e4fb22c20,con_003,met_003,...,3,pla_003,pla_003,TikTok,https://www.tiktok.com,Amanda Brooks,(555) 345-6789,amanda.brooks@marketflash.com,ad_005,3
3,4,Campaign 4,2023-02-23,2023-03-09,46782.3125,37425.85,emp_003,8053dbe1-1ffb-4433-afdf-1b5402563f02,con_004,met_004,...,4,pla_004,pla_004,Instagram,https://www.instagram.com,Daniel Lee,(555) 456-7890,daniel.lee@marketflash.com,ad_002,4
4,5,Campaign 5,2023-11-20,2023-12-11,60737.9250,48590.34,emp_004,0862cce9-4848-4e3d-a46f-e4bdfef5522f,con_005,met_005,...,5,pla_004,pla_004,Instagram,https://www.instagram.com,Daniel Lee,(555) 456-7890,daniel.lee@marketflash.com,ad_002,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Campaign 996,2023-02-15,2023-03-13,15466.7750,12373.42,emp_006,b8f81c35-b89a-4991-a94c-09dca8b51857,con_996,met_996,...,996,pla_004,pla_004,Instagram,https://www.instagram.com,Daniel Lee,(555) 456-7890,daniel.lee@marketflash.com,ad_002,4
996,997,Campaign 997,2023-05-14,2023-06-01,60914.9500,48731.96,emp_008,e601de7a-34c1-4b1f-8413-f25b8fa978c9,con_997,met_997,...,997,pla_005,pla_005,Facebook,https://www.facebook.com,Emily Carter,(555) 567-8901,emily.carter@marketflash.com,ad_004,5
997,998,Campaign 998,2023-01-15,2023-02-13,56578.9750,45263.18,emp_008,b5d2a46c-a7de-4939-b3df-0161d8be8f24,con_998,met_998,...,998,pla_005,pla_005,Facebook,https://www.facebook.com,Emily Carter,(555) 567-8901,emily.carter@marketflash.com,ad_004,5
998,999,Campaign 999,2023-01-30,2023-01-31,8074.6750,6459.74,emp_002,5fabe2aa-ee3c-4a8e-ac35-ff6d66f24c59,con_999,met_999,...,999,pla_003,pla_003,TikTok,https://www.tiktok.com,Amanda Brooks,(555) 345-6789,amanda.brooks@marketflash.com,ad_005,3


In [24]:
# Join clients and campaigns

query = """
SELECT
  ca.campaign_id,
  ca.campaign_name,
  ca.campaign_cost,
  ca.client_id,
  cl.client_name,
  cl.client_contact_person
FROM
  `snappy-rainfall-450911-k6.market_flash_stage.campaigns` ca
LEFT JOIN
  `snappy-rainfall-450911-k6.market_flash_stage.clients` cl
ON
  ca.client_id = cl.client_id
WHERE
  ca.campaign_id IS NOT NULL
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,campaign_id,campaign_name,campaign_cost,client_id,client_name,client_contact_person
0,1,Campaign 1,13961.03,cli_001,Lopez PLC,Barbara Walker
1,2,Campaign 2,43804.31,cli_002,"Weaver, Garner and Ramos",Melinda Johnston
2,3,Campaign 3,36007.47,cli_003,Salinas-Chavez,Chelsea Hoffman
3,4,Campaign 4,37425.85,cli_004,"Russell, Wilson and Rogers",Michael Howard
4,5,Campaign 5,48590.34,cli_005,White Ltd,Nathan Weber
...,...,...,...,...,...,...
995,996,Campaign 996,12373.42,cli_059,"Rowe, Morse and Morgan",Destiny Atkins
996,997,Campaign 997,48731.96,cli_026,"Bradley, Jackson and Miller",Dr. David Glass Jr.
997,998,Campaign 998,45263.18,cli_052,Hernandez Inc,Traci Holden
998,999,Campaign 999,6459.74,cli_046,"Holloway, Cox and Ford",Dennis Carr


In [25]:
#Calc sum and avg campaign_cost using new table created above

query = """
SELECT
  client_id,
  client_name,
  SUM(campaign_cost) AS sum_costs,
  AVG(campaign_cost) AS avg_costs,
  COUNT(*) AS total_campaigns
FROM
  `snappy-rainfall-450911-k6.market_flash_integration.campaign_client_joined_table`
GROUP BY
  client_id,
  client_name
ORDER BY
  total_campaigns DESC
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,client_id,client_name,sum_costs,avg_costs,total_campaigns
0,cli_039,Cooper-Alexander,320742.98,16037.149000,20
1,cli_046,"Holloway, Cox and Ford",452950.74,22647.537000,20
2,cli_034,Anderson Inc,490918.01,25837.790000,19
3,cli_003,Salinas-Chavez,464442.15,25802.341667,18
4,cli_006,Patterson and Sons,591591.01,32866.167222,18
...,...,...,...,...,...
73,cli_004,"Russell, Wilson and Rogers",218354.63,31193.518571,7
74,cli_053,"Wheeler, Garcia and Johnson",187868.00,31311.333333,6
75,cli_060,"Mcclain, Beltran and Patterson",160578.07,26763.011667,6
76,cli_074,"Salinas, Rodriguez and Bowman",129290.50,21548.416667,6


In [26]:
# Join the campaigns table with clients and payments to see what client has paid what money over

query = """

SELECT
  ca.campaign_id,
  ca.campaign_name,
  ca.start_date,
  ca.end_date,
  ca.campaign_cost,
  cl.client_id,
  cl.client_name,
  pm.pmt_id,
  pmt_date,
  amount
FROM
  `snappy-rainfall-450911-k6.market_flash_stage.campaigns` ca
LEFT JOIN
  `snappy-rainfall-450911-k6.market_flash_stage.clients` cl
ON
  ca.client_id = cl.client_id
LEFT JOIN
  `snappy-rainfall-450911-k6.market_flash_stage.payments` pm
ON
  ca.pmt_id = pm.pmt_id
WHERE
  ca.campaign_id IS NOT NULL
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,campaign_id,campaign_name,start_date,end_date,campaign_cost,client_id,client_name,pmt_id,pmt_date,amount
0,1,Campaign 1,2023-12-18,2024-01-10,13961.03,cli_001,Lopez PLC,ff111cd7-9a9b-4cff-a713-4a32e044212a,2024-11-10,2965
1,2,Campaign 2,2023-10-12,2023-11-09,43804.31,cli_002,"Weaver, Garner and Ramos",0b415dd8-d6d7-46d1-a79c-213170956edf,2025-01-05,626
2,3,Campaign 3,2023-05-18,2023-06-04,36007.47,cli_003,Salinas-Chavez,9e15fb41-4f9f-4699-a53e-924e4fb22c20,2025-03-23,3642
3,4,Campaign 4,2023-02-23,2023-03-09,37425.85,cli_004,"Russell, Wilson and Rogers",8053dbe1-1ffb-4433-afdf-1b5402563f02,2025-01-30,4826
4,5,Campaign 5,2023-11-20,2023-12-11,48590.34,cli_005,White Ltd,0862cce9-4848-4e3d-a46f-e4bdfef5522f,2025-01-12,4680
...,...,...,...,...,...,...,...,...,...,...
995,996,Campaign 996,2023-02-15,2023-03-13,12373.42,cli_059,"Rowe, Morse and Morgan",b8f81c35-b89a-4991-a94c-09dca8b51857,2024-08-31,4520
996,997,Campaign 997,2023-05-14,2023-06-01,48731.96,cli_026,"Bradley, Jackson and Miller",e601de7a-34c1-4b1f-8413-f25b8fa978c9,2025-04-09,2629
997,998,Campaign 998,2023-01-15,2023-02-13,45263.18,cli_052,Hernandez Inc,b5d2a46c-a7de-4939-b3df-0161d8be8f24,2025-01-25,596
998,999,Campaign 999,2023-01-30,2023-01-31,6459.74,cli_046,"Holloway, Cox and Ford",5fabe2aa-ee3c-4a8e-ac35-ff6d66f24c59,2025-03-23,1389


In [27]:
# SUM payments for each client for each campaign

query = """
SELECT
  client_id,
  client_name,
  campaign_id,
  campaign_name,
  SUM(amount) AS total_payment_amt,
FROM
  `snappy-rainfall-450911-k6.market_flash_integration.campaign_client_payments_table`
GROUP BY
  client_id,
  client_name,
  campaign_id,
  campaign_name
ORDER BY
  client_id,
  campaign_id
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,client_id,client_name,campaign_id,campaign_name,total_payment_amt
0,cli_001,Lopez PLC,1,Campaign 1,2965
1,cli_001,Lopez PLC,16,Campaign 16,2431
2,cli_001,Lopez PLC,25,Campaign 25,4888
3,cli_001,Lopez PLC,51,Campaign 51,3739
4,cli_001,Lopez PLC,108,Campaign 108,4531
...,...,...,...,...,...
995,cli_078,Griffin LLC,509,Campaign 509,4175
996,cli_078,Griffin LLC,517,Campaign 517,2003
997,cli_078,Griffin LLC,844,Campaign 844,2905
998,cli_078,Griffin LLC,887,Campaign 887,2001


In [28]:
# Sum payments per client

query = """
SELECT client_id, client_name, sum(total_payment_amt) AS total_paid
FROM `snappy-rainfall-450911-k6.market_flash_integration.campaign_client_sum_amts_paid`
GROUP BY client_id, client_name
ORDER BY total_paid DESC
"""
# Run the query and store results in DataFrame
df = client.query(query).to_dataframe()

# Display the results
df

,client_id,client_name,total_paid
0,cli_046,"Holloway, Cox and Ford",62720
1,cli_034,Anderson Inc,57961
2,cli_026,"Bradley, Jackson and Miller",54993
3,cli_001,Lopez PLC,52328
4,cli_039,Cooper-Alexander,49033
...,...,...,...
73,cli_066,Roberts-Howe,17531
74,cli_074,"Salinas, Rodriguez and Bowman",14635
75,cli_072,"Hernandez, Keller and Berry",12856
76,cli_060,"Mcclain, Beltran and Patterson",10753
